# Text generator based on RNN

### Import libraries

In [1]:
import tensorflow as tf
import numpy as np

### Configurations

In [2]:
vocab = (" $%'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            "\\^_abcdefghijklmnopqrstuvwxyz{|}")
TestText="Too young, too simple, sometimes naive. "
graphPath=r".\graphs"
num_step=len(TestText)-1
state_size=100#Make it simple
batchSize=1
def Encode(string,name=None):
    tensor=[vocab.find(ch)+1 for ch in string]
    tensor = tf.one_hot(tensor, depth=len(vocab)+1, on_value=1.0, off_value=0.0, axis=-1, dtype=tf.float32, name=name)
    with tf.Session() as sess:
        nparray=tensor.eval()
    return nparray
def Decode(nparray):
    return "".join([vocab[index-1] if index>0 else "[INVALID]" for index in np.argmax(nparray,axis=1)])
X=Encode(TestText)
Y=X[1:]
X=X[:-1]
print(X.shape,Y.shape)

(39, 84) (39, 84)


### Create graph

In [3]:
with tf.variable_scope("RNN") as modelScope:
    with tf.variable_scope("structure") as structureScope:
        inputs=tf.placeholder(dtype=tf.float32,shape=[None,len(vocab)+1])
        outputs=tf.placeholder(dtype=tf.float32,shape=[None,len(vocab)+1])
        cell=tf.contrib.rnn.GRUCell(num_units=state_size)
        w=tf.get_variable(name="Weight_s_o",shape=[state_size,len(vocab)+1],initializer=tf.truncated_normal_initializer())
        b=tf.get_variable(name="Bias_s_o",shape=[1,len(vocab)+1],initializer=tf.truncated_normal_initializer())
        initState=cell.zero_state(batch_size=batchSize,dtype=tf.float32)
        state=initState
        netOutputs=[]
        #Since the tf.nn.softmax_cross_entropy_with_logits() runs softmax internally, and 
        #I don't find another function to compute the cross entropy without internal softmax
        #operations, so...
        netVectsForLoss=[]
        for i in range(num_step):
            if(i>0):
                structureScope.reuse_variables()
            input_=tf.slice(input_=inputs,begin=[i,0],size=[1,-1])
            output,state=cell(inputs=input_,state=state)
            output=tf.add(tf.matmul(output,w),b)
            netVectsForLoss.append(output)
            output=tf.nn.softmax(output,name="Output")
            netOutputs.append(output)
            input_=output
        netOutputs=tf.concat(values=netOutputs,axis=0)
        netVectsForLoss=tf.concat(values=netVectsForLoss,axis=0,name="vectForLoss")
    with tf.name_scope("training"):
        loss=tf.losses.softmax_cross_entropy(logits=netVectsForLoss,onehot_labels=outputs)
        loss=tf.reduce_mean(loss,name="loss")
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001,name="optimizer").minimize(loss)
    with tf.name_scope("summary") as scope:
        summary_op=tf.summary.merge_all()

In [4]:
with tf.Session() as sess:
    writer=tf.summary.FileWriter(logdir=graphPath,graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    result=sess.run(netOutputs,{inputs:X,outputs:Y})
print(Decode(result))

+++m.''....cc'c'''''''''''''..c'cc..ccc


In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        sess.run(optimizer,feed_dict={inputs:X,outputs:Y})
    result=sess.run(netOutputs,{inputs:X})
print(Decode(result))

  young, too simple, sometimes naive. n
